### Fruit Recognition

Stefano Nicolis, Pietro Turco

### 1. Libraries import

In [19]:
import csv
from PIL import Image as im
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.svm import SVC
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset
import torch

### 2. Dataset import

In [35]:
data_set_path = 'C:\\F_drive\\Ste\\UNIVERSITÀ\\Magistrale\\ML&AI\\images\\selected_datasets\\dataset.csv'

data_set = {} # { label : # images per label }
"""
with open(data_set_path, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)

    for row in csv_reader:
        key = (int)(row[0]) + 1
        if(key in data_set.keys()):
            data_set[key] += 1
        else:            
            data_set[key] = 1

# Data to plot
labels = []
data = []

for x, y in data_set.items():
    labels.append(x)
    data.append(y)

# Plot
plt.pie(data, labels=labels)

plt.axis('equal')
plt.show()
"""

data = np.loadtxt(open(data_set_path, "rb"), delimiter=",", dtype = int, skiprows=0)
# Extract label column
labels = data[:, 0]
# Remove label column 
data = data[:,1:]

# Split dataset into training and test, 80%/20%
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size = 0.2)
print(test_data.shape)
print(train_data.shape)

# Converte i dati e le etichette da NumPy a PyTorch Tensor
train_data_cnn = torch.from_numpy(train_data).float()
train_labels_cnn = torch.from_numpy(train_labels).long()
test_data_cnn = torch.from_numpy(test_data).float()
test_labels_cnn = torch.from_numpy(test_labels).long()

(2000, 14700)
(8000, 14700)


### PCA

In [10]:
# Create reduced data sets with PCA
pca = PCA(n_components = 140)
train_data_cnn_pca = pca.fit_transform(train_data)
test_data_pca = pca.transform(test_data)

# Check the variances of the 40 selected components.
X_train_var = np.var(train_data_pca)
X_train_var = sum(pca.explained_variance_ratio_)
print("X_train_var after PCA with 1400 components: "+str(X_train_var))

pca = PCA(n_components = 15)
train_data_pca = pca.fit_transform(train_data)
test_data_pca = pca.transform(test_data)

# Check the variances of the 40 selected components.
X_train_var = np.var(train_data_pca)
X_train_var = sum(pca.explained_variance_ratio_)
print("X_train_var after PCA with 15 components: "+str(X_train_var))

pca = PCA(n_components = 3)
train_data_pca = pca.fit_transform(train_data)
test_data_pca = pca.transform(test_data)

# Check the variances of the 40 selected components.
X_train_var = np.var(train_data_pca)
X_train_var = sum(pca.explained_variance_ratio_)
print("X_train_var after PCA with 3 components: "+str(X_train_var))


X_train_var after PCA with 1400 components: 0.9637198432596605
X_train_var after PCA with 15 components: 0.8229459330729011
X_train_var after PCA with 3 components: 0.5731766619235649


### 3. KNN, no PCA

In [3]:

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(train_data, train_labels)

result = knn.predict(test_data)

accuracy = accuracy_score(result, test_labels)
print(f'Accuracy: {accuracy}')

"""
import random

SIZE = 1000
random_data = []
random_labels = []

# Generate random dataset
for x in range(SIZE):
    r = random.randint(0, 9999)
    random_data.append(data[r])
    random_labels.append(labels[r])

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(random_data, random_labels)

for x in range(5):
    r = random.randint(0, SIZE)
    print(knn.predict([random_data[r]]))    
    print(random_labels[r])
    print("=======")
"""

Accuracy: 0.999


'\nimport random\n\nSIZE = 1000\nrandom_data = []\nrandom_labels = []\n\n# Generate random dataset\nfor x in range(SIZE):\n    r = random.randint(0, 9999)\n    random_data.append(data[r])\n    random_labels.append(labels[r])\n\nknn = KNeighborsClassifier(n_neighbors=3)\nknn.fit(random_data, random_labels)\n\nfor x in range(5):\n    r = random.randint(0, SIZE)\n    print(knn.predict([random_data[r]]))    \n    print(random_labels[r])\n    print("=======")\n'

### 4. KNN, PCA

In [18]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(train_data_pca, train_labels)

result = knn.predict(test_data_pca)

accuracy = accuracy_score(result, test_labels)
print(f'Accuracy: {accuracy}')

### 5. SVM

In [6]:
KF = ['linear', 'poly', 'rbf', 'sigmoid']

svm_classifier = SVC(kernel='linear', decision_function_shape='ovr')
print("created SVM C")
svm_classifier.fit(train_data, train_labels)
print("Fitted data")
svm_pred = svm_classifier.predict(test_data)
print("predicted")

svm_accuracy = accuracy_score(test_labels, svm_pred)
svm_precision = precision_score(test_labels, svm_pred, average='macro')
svm_recall = recall_score(test_labels, svm_pred, average='macro')
svm_cm = confusion_matrix(test_labels, svm_pred)

print("\nSupport Vector Machines (SVM), KERNEL FUNCTION: " + 'linear' + ", Results:")
print("Accuracy:", svm_accuracy)
print("Precision:", svm_precision)
print("Recall:", svm_recall)
    

created SVM C
Fitted data
predicted

Support Vector Machines (SVM), KERNEL FUNCTION: linear, Results:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0


### 6. SVM, PCA

In [8]:
KF = ['linear', 'poly', 'rbf', 'sigmoid']

svm_classifier = SVC(kernel='linear', decision_function_shape='ovr')  # TODO studia ovr
print("created SVM C")
svm_classifier.fit(train_data_pca, train_labels)
print("Fitted data")
svm_pred = svm_classifier.predict(test_data_pca)
print("predicted")

svm_accuracy = accuracy_score(test_labels, svm_pred)
svm_precision = precision_score(test_labels, svm_pred, average='macro')
svm_recall = recall_score(test_labels, svm_pred, average='macro')
svm_cm = confusion_matrix(test_labels, svm_pred)

print("\nSupport Vector Machines (SVM), KERNEL FUNCTION: " + 'linear' + ", Results:")
print("Accuracy:", svm_accuracy)
print("Precision:", svm_precision)
print("Recall:", svm_recall)
    

created SVM C
Fitted data
predicted

Support Vector Machines (SVM), KERNEL FUNCTION: linear, Results:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0


### .7 CNN

In [38]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(in_features=16384, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=num_classes)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = self.pool1(x)
        x = nn.functional.relu(self.conv2(x))
        x = self.pool2(x)
        x = self.flatten(x)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create an instance of the CNN model
model = CNN(num_classes=25)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

# Train the model for a number of epochs
num_epochs = 20
batch_size = 64
for epoch in range(num_epochs):
    # Shuffle the training data at the start of each epoch
    permutation = np.random.permutation(len(train_data_cnn))
    train_data_cnn = train_data_cnn[permutation]
    train_labels_cnn = train_labels_cnn[permutation]

    # Split the training data into batches
    num_batches = len(train_data_cnn) // batch_size
    for batch_idx in range(num_batches):
        # Select the current batch of data and labels
        start = batch_idx * batch_size
        end = start + batch_size
        data, target = train_data_cnn[start:end], train_labels_cnn[start:end]

        data = data.view(-1, 3, 70, 70)

        # Zero gradients
        optimizer.zero_grad()

        # Make a prediction with the model
        output = model(data)

        # Calculate loss
        loss = criterion(output, target)

        # Perform backpropagation to calculate gradients
        loss.backward()

        # Update model weights
        optimizer.step()

    # Evaluate the model on the test set
    correct = 0
    total = 0
    with torch.no_grad():
        data, target = test_data_cnn, test_labels_cnn
        data = data.view(-1, 3, 70, 70)
        output = model(data)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()
    accuracy = 100 * correct / total
    print(f'Epoch: {epoch + 1}/{num_epochs}, Test Accuracy: {accuracy:.2f}%')

Epoch: 1/20, Test Accuracy: 98.40%
Epoch: 2/20, Test Accuracy: 98.50%
Epoch: 3/20, Test Accuracy: 99.60%
Epoch: 4/20, Test Accuracy: 96.80%
Epoch: 5/20, Test Accuracy: 99.95%
Epoch: 6/20, Test Accuracy: 99.75%
Epoch: 7/20, Test Accuracy: 87.25%
Epoch: 8/20, Test Accuracy: 99.90%
Epoch: 9/20, Test Accuracy: 100.00%
Epoch: 10/20, Test Accuracy: 100.00%
Epoch: 11/20, Test Accuracy: 100.00%
Epoch: 12/20, Test Accuracy: 100.00%
Epoch: 13/20, Test Accuracy: 100.00%
Epoch: 14/20, Test Accuracy: 100.00%
Epoch: 15/20, Test Accuracy: 100.00%


KeyboardInterrupt: 